<a href="https://colab.research.google.com/github/nalinichandrasekhar/customer-analysis/blob/main/Ecommerce_Customer_Behaviour_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Scenario**

 Ecommerce company wants to understand its customer behavious better so that they can increase sales and customer satisfavtion.They also need to understand customer behaviour to come up with marketing strategies to drive sales.This can help in data driven decision making for driving an increase in sales and customer delight.

**Questions to be answered**


*   Factors that influence customer spending
*   Impact of membership types
*   Any patterns in customer satisfaction?
*   Most valuble customer segments

**Loading the data set**

In [50]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [51]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

df=pd.read_csv('ecommerce_data.csv')
df.head()

,customer_id,age,gender,membership_type,annual_income,total_purchases,avg_order_value,customer_satisfaction,time_on_site_minutes,city,total_spent
0,1,56,Female,Premium,"$68,070.89",9,$67.28,$5.00,$9.80,Los Angeles,$901.27
1,2,69,Female,VIP,"$40,657.50",2,$26.60,$5.00,$8.10,Phoenix,$79.31
2,3,46,Female,Basic,"$54,391.14",11,$102.50,$4.00,$1.60,Chicago,"$1,500.83"
3,4,32,Female,Basic,"$108,556.76",8,$136.72,$4.00,$3.50,Los Angeles,"$1,891.62"
4,5,60,Male,Premium,"$98,188.58",6,$48.79,$4.00,$8.20,Chicago,$485.33


**Checking the basic information in the dataset to get an overview and understand the data**

The Column names are assesed to understand the data

In [7]:
df.shape

(500, 11)

In [8]:
df.columns

Index(['customer_id', 'age', 'gender', 'membership_type', 'annual_income',
       'total_purchases', 'avg_order_value', 'customer_satisfaction',
       'time_on_site_minutes', 'city', 'total_spent'],
      dtype='object')

In [10]:
df.info(verbose=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Columns: 11 entries, customer_id to total_spent
dtypes: float64(5), int64(3), object(3)
memory usage: 43.1+ KB


The df.info() gives the range,no of columns and the data types involved in the dataset.

In [16]:
df.isna()
df.isna().sum()


,0
customer_id,0
age,0
gender,0
membership_type,0
annual_income,0
total_purchases,0
avg_order_value,0
customer_satisfaction,15
time_on_site_minutes,0
city,0


There are 15 missing values in the customer_satisfaction column

In [17]:
df.isna().mean() * 100

,0
customer_id,0.0
age,0.0
gender,0.0
membership_type,0.0
annual_income,0.0
total_purchases,0.0
avg_order_value,0.0
customer_satisfaction,3.0
time_on_site_minutes,0.0
city,0.0


These 15 values attribute to 3 percent of the customer satisfaction data.Since its only 3% we can fill them with the median value.

In [22]:
df["customer_satisfaction"] = df["customer_satisfaction"].fillna(
    df["customer_satisfaction"].median()
)

Now we have to check for duplicate rows if any

In [28]:
df.duplicated()


,0
0,False
1,False
2,False
3,False
4,False
...,...
495,False
496,False
497,False
498,False


Since there are no duplicated rows then we move on to check the outliers.

In [36]:
def detect_outliers(column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1

    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR

    return df[(df[column] < lower) | (df[column] > upper)]


The above code creates a function that computes the q1 and q3 and the the interquartile range.It also defines the lower and the upper bounds to filter values outside the range to detect outliers.

In [33]:
outliers = detect_outliers("total_spent")
print(len(outliers))
count = len(outliers)
percentage = (count / len(df)) * 100

print(f"Outliers: {count}")
print(f"Percentage: {percentage:.2f}%")

19
Outliers: 19
Percentage: 3.80%


In [34]:
outliers = detect_outliers("time_on_site_minutes")
print(len(outliers))
count = len(outliers)
percentage = (count / len(df)) * 100

print(f"Outliers: {count}")
print(f"Percentage: {percentage:.2f}%")

13
Outliers: 13
Percentage: 2.60%


In [35]:

outliers = detect_outliers("annual_income")
print(len(outliers))
count = len(outliers)
percentage = (count / len(df)) * 100

print(f"Outliers: {count}")
print(f"Percentage: {percentage:.2f}%")

0
Outliers: 0
Percentage: 0.00%


So the outliers could be system error or rare data.This shows that the data is not skewed or heavy tailed.

**Univariate Analysis**

Analyse the Numerical and the categorical variables.Understanding the columns individually.

In [43]:
df.describe()

,customer_id,age,annual_income,total_purchases,avg_order_value,customer_satisfaction,time_on_site_minutes,total_spent
count,$500.00,$500.00,$500.00,$500.00,$500.00,$500.00,$500.00,$500.00
mean,$250.50,$44.22,"$58,066.75",$7.85,$72.48,$3.72,$5.91,$825.01
std,$144.48,$15.04,"$23,861.87",$2.69,$29.33,$1.01,$4.30,$523.38
min,$1.00,$18.00,"$20,000.00",$1.00,$10.00,$1.00,$0.30,$56.45
25%,$125.75,$32.00,"$39,833.39",$6.00,$52.87,$3.00,$2.80,$451.27
50%,$250.50,$45.00,"$56,943.21",$8.00,$73.11,$4.00,$4.70,$747.29
75%,$375.25,$57.00,"$73,417.05",$10.00,$93.45,$4.00,$8.30,"$1,059.05"
max,$500.00,$69.00,"$120,042.08",$16.00,$151.54,$5.00,$25.40,"$3,816.42"




---



In [44]:
df["total_spent"].isna().mean() * 100

np.float64(0.0)

The following  key observations are made:


*   Age-Ranges from 18 to 69.Mean is 44 which is middle age customer base.
*   the average income mean and median are closer suggesting normal distribution


